<a href="https://colab.research.google.com/github/Subramaniya-pillai/data_engineering/blob/main/hr_analytics_tasks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Task 1: Ingestion & Exploration

In [9]:

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("HR Analytics").getOrCreate()

# Load CSV and JSON files
emp_df = spark.read.option("header", True).option("inferSchema", True).csv("/content/employees (1).csv")



In [10]:
att_df = spark.read.option("header", True).option("inferSchema", True).csv("/content/attendance.csv")
bonus_df = spark.read.option("multiline", True).json("bonuses.json")

# Show schemas and samples
emp_df.printSchema()
emp_df.show()

att_df.printSchema()
att_df.show()

bonus_df.printSchema()
bonus_df.show()

# Count distinct departments
emp_df.select("Department").distinct().count()

root
 |-- EmpID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Department: string (nullable = true)
 |-- JoinDate: date (nullable = true)
 |-- Salary: integer (nullable = true)
 |-- ManagerID: double (nullable = true)

+-----+------+-----------+----------+------+---------+
|EmpID|  Name| Department|  JoinDate|Salary|ManagerID|
+-----+------+-----------+----------+------+---------+
|    1| Anita|         HR|2021-05-01| 55000|     NULL|
|    2|   Raj|Engineering|2020-03-15| 80000|      1.0|
|    3|Simran|Engineering|2022-07-10| 75000|      1.0|
|    4| Aamir|  Marketing|2019-11-20| 60000|      1.0|
|    5| Nisha|         HR|2023-01-05| 50000|      1.0|
+-----+------+-----------+----------+------+---------+

root
 |-- EmpID: integer (nullable = true)
 |-- Date: date (nullable = true)
 |-- Status: string (nullable = true)

+-----+----------+-------+
|EmpID|      Date| Status|
+-----+----------+-------+
|    1|2024-04-01|Present|
|    1|2024-04-02|Present|
|    2|2024-0

3

## Task 2: DataFrame Operations

In [11]:

from pyspark.sql.functions import datediff, current_date, round, col

# Add TenureYears
emp_df = emp_df.withColumn("TenureYears", round(datediff(current_date(), col("JoinDate")) / 365, 2))
emp_df.select("EmpID", "Name", "TenureYears").show()

# TotalCompensation
emp_bonus_df = emp_df.join(bonus_df, "EmpID").withColumn("TotalCompensation", col("Salary") + col("Bonus"))
emp_bonus_df.select("EmpID", "Name", "TotalCompensation").show()

# More than 2 years
emp_bonus_df.filter(col("TenureYears") > 2).show()

# With Manager
emp_df.filter(col("ManagerID").isNotNull()).show()


+-----+------+-----------+
|EmpID|  Name|TenureYears|
+-----+------+-----------+
|    1| Anita|       4.11|
|    2|   Raj|       5.24|
|    3|Simran|       2.92|
|    4| Aamir|       5.56|
|    5| Nisha|       2.43|
+-----+------+-----------+

+-----+------+-----------------+
|EmpID|  Name|TotalCompensation|
+-----+------+-----------------+
|    1| Anita|            60000|
|    2|   Raj|            87000|
|    3|Simran|            81500|
|    4| Aamir|            66000|
|    5| Nisha|            54000|
+-----+------+-----------------+

+-----+------+-----------+----------+------+---------+-----------+-----+----+-----------------+
|EmpID|  Name| Department|  JoinDate|Salary|ManagerID|TenureYears|Bonus|Year|TotalCompensation|
+-----+------+-----------+----------+------+---------+-----------+-----+----+-----------------+
|    1| Anita|         HR|2021-05-01| 55000|     NULL|       4.11| 5000|2023|            60000|
|    2|   Raj|Engineering|2020-03-15| 80000|      1.0|       5.24| 7000|20

## Task 3: Aggregation

In [12]:

# Avg salary per department
emp_df.groupBy("Department").avg("Salary").show()

# Employees under each manager
emp_df.groupBy("ManagerID").count().withColumnRenamed("count", "NumEmployees").show()

# Absences per employee
att_df.filter(col("Status") == "Absent").groupBy("EmpID").count().withColumnRenamed("count", "AbsenceCount").show()


+-----------+-----------+
| Department|avg(Salary)|
+-----------+-----------+
|Engineering|    77500.0|
|         HR|    52500.0|
|  Marketing|    60000.0|
+-----------+-----------+

+---------+------------+
|ManagerID|NumEmployees|
+---------+------------+
|     NULL|           1|
|      1.0|           4|
+---------+------------+

+-----+------------+
|EmpID|AbsenceCount|
+-----+------------+
|    4|           2|
|    2|           1|
+-----+------------+



## Task 4: Joins

In [13]:

from pyspark.sql.functions import count, sum, when

# Attendance %
att_pct_df = att_df.groupBy("EmpID").agg(
    count("*").alias("TotalDays"),
    sum(when(col("Status") == "Present", 1).otherwise(0)).alias("PresentDays")
).withColumn("AttendancePct", round(col("PresentDays") / col("TotalDays") * 100, 2))

emp_att_pct_df = emp_df.join(att_pct_df, "EmpID")
emp_att_pct_df.select("EmpID", "Name", "AttendancePct").show()

# Top 3 by compensation
emp_bonus_df.orderBy(col("TotalCompensation").desc()).limit(3).show()

# Multi-level join
multi_df = emp_df.join(bonus_df, "EmpID").join(att_df, "EmpID")
multi_df.show()


+-----+------+-------------+
|EmpID|  Name|AttendancePct|
+-----+------+-------------+
|    1| Anita|        100.0|
|    3|Simran|        100.0|
|    5| Nisha|        100.0|
|    4| Aamir|          0.0|
|    2|   Raj|         50.0|
+-----+------+-------------+

+-----+------+-----------+----------+------+---------+-----------+-----+----+-----------------+
|EmpID|  Name| Department|  JoinDate|Salary|ManagerID|TenureYears|Bonus|Year|TotalCompensation|
+-----+------+-----------+----------+------+---------+-----------+-----+----+-----------------+
|    2|   Raj|Engineering|2020-03-15| 80000|      1.0|       5.24| 7000|2023|            87000|
|    3|Simran|Engineering|2022-07-10| 75000|      1.0|       2.92| 6500|2023|            81500|
|    4| Aamir|  Marketing|2019-11-20| 60000|      1.0|       5.56| 6000|2023|            66000|
+-----+------+-----------+----------+------+---------+-----------+-----+----+-----------------+

+-----+------+-----------+----------+------+---------+-----------

## Task 5: String & Date Functions

In [14]:

from pyspark.sql.functions import year, month, regexp_replace, lpad, lit, concat

# Extract year and month
emp_df = emp_df.withColumn("JoinYear", year("JoinDate")).withColumn("JoinMonth", month("JoinDate"))
emp_df.select("EmpID", "JoinDate", "JoinYear", "JoinMonth").show()

# Mask names
emp_df = emp_df.withColumn("MaskedName", regexp_replace("Name", "(?<=.).", "*"))
emp_df.select("EmpID", "Name", "MaskedName").show()

# EmpCode
emp_df = emp_df.withColumn("EmpCode", lpad(col("EmpID").cast("string"), 3, "0"))
emp_df = emp_df.withColumn("EmpCode", concat(lit("EMP"), col("EmpCode")))
emp_df.select("EmpID", "EmpCode").show()


+-----+----------+--------+---------+
|EmpID|  JoinDate|JoinYear|JoinMonth|
+-----+----------+--------+---------+
|    1|2021-05-01|    2021|        5|
|    2|2020-03-15|    2020|        3|
|    3|2022-07-10|    2022|        7|
|    4|2019-11-20|    2019|       11|
|    5|2023-01-05|    2023|        1|
+-----+----------+--------+---------+

+-----+------+----------+
|EmpID|  Name|MaskedName|
+-----+------+----------+
|    1| Anita|     A****|
|    2|   Raj|       R**|
|    3|Simran|    S*****|
|    4| Aamir|     A****|
|    5| Nisha|     N****|
+-----+------+----------+

+-----+-------+
|EmpID|EmpCode|
+-----+-------+
|    1| EMP001|
|    2| EMP002|
|    3| EMP003|
|    4| EMP004|
|    5| EMP005|
+-----+-------+



## Task 6: Conditional & Null Handling

In [15]:

from pyspark.sql.functions import when

# Performance label
bonus_df = bonus_df.withColumn("Performance",
    when(col("Bonus") > 6000, "High")
    .when((col("Bonus") >= 4000) & (col("Bonus") <= 6000), "Medium")
    .otherwise("Low")
)
bonus_df.select("EmpID", "Bonus", "Performance").show()

# Null handling
emp_df = emp_df.fillna({"ManagerID": "No Manager"})
emp_df.select("EmpID", "ManagerID").show()


+-----+-----+-----------+
|EmpID|Bonus|Performance|
+-----+-----+-----------+
|    1| 5000|     Medium|
|    2| 7000|       High|
|    3| 6500|       High|
|    4| 6000|     Medium|
|    5| 4000|     Medium|
+-----+-----+-----------+

+-----+---------+
|EmpID|ManagerID|
+-----+---------+
|    1|     NULL|
|    2|      1.0|
|    3|      1.0|
|    4|      1.0|
|    5|      1.0|
+-----+---------+



## Task 7: Spark SQL

In [16]:

spark.sql("CREATE DATABASE IF NOT EXISTS hr")
spark.catalog.setCurrentDatabase("hr")

emp_df.write.mode("overwrite").saveAsTable("employees")
att_df.write.mode("overwrite").saveAsTable("attendance")
bonus_df.write.mode("overwrite").saveAsTable("bonuses")

# Queries
spark.sql("""
    SELECT Department, Name, Salary FROM employees e
    WHERE Salary = (SELECT MAX(Salary) FROM employees e2 WHERE e.Department = e2.Department)
""").show()

spark.sql("""
    SELECT e.Department, ROUND(SUM(CASE WHEN a.Status = 'Present' THEN 1 ELSE 0 END)/COUNT(*), 2) AS AttendanceRate
    FROM employees e JOIN attendance a ON e.EmpID = a.EmpID
    GROUP BY e.Department
""").show()

spark.sql("""
    SELECT * FROM employees
    WHERE JoinDate > '2021-01-01' AND Salary > 70000
""").show()


+-----------+-----+------+
| Department| Name|Salary|
+-----------+-----+------+
|         HR|Anita| 55000|
|Engineering|  Raj| 80000|
|  Marketing|Aamir| 60000|
+-----------+-----+------+

+-----------+--------------+
| Department|AttendanceRate|
+-----------+--------------+
|Engineering|          0.75|
|         HR|           1.0|
|  Marketing|           0.0|
+-----------+--------------+

+-----+------+-----------+----------+------+---------+-----------+--------+---------+----------+-------+
|EmpID|  Name| Department|  JoinDate|Salary|ManagerID|TenureYears|JoinYear|JoinMonth|MaskedName|EmpCode|
+-----+------+-----------+----------+------+---------+-----------+--------+---------+----------+-------+
|    3|Simran|Engineering|2022-07-10| 75000|      1.0|       2.92|    2022|        7|    S*****| EMP003|
+-----+------+-----------+----------+------+---------+-----------+--------+---------+----------+-------+



## Task 8: Advanced (Optional)

In [17]:

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# UDF to classify departments
def classify_dept(dept):
    return "Tech" if dept == "Engineering" else "Non-Tech"

dept_udf = udf(classify_dept, StringType())
emp_df = emp_df.withColumn("DeptCategory", dept_udf("Department"))
emp_df.select("EmpID", "Department", "DeptCategory").show()

# Create view and save as Parquet
emp_attendance_summary = emp_df.join(att_pct_df, "EmpID")
emp_attendance_summary.createOrReplaceTempView("emp_attendance_summary")
emp_attendance_summary.write.mode("overwrite").partitionBy("Department").parquet("output/emp_attendance_summary")


+-----+-----------+------------+
|EmpID| Department|DeptCategory|
+-----+-----------+------------+
|    1|         HR|    Non-Tech|
|    2|Engineering|        Tech|
|    3|Engineering|        Tech|
|    4|  Marketing|    Non-Tech|
|    5|         HR|    Non-Tech|
+-----+-----------+------------+

